In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [3]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_bu = pd.read_pickle(ENV.bureau_cleaned.value)
print('Installment shape: {}'.format(X_bu.shape))

X_bu_fe = pd.read_pickle(ENV.bureau_cleaned_rnnALL.value)
print('Installment shape: {}'.format(X_bu_fe.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Installment shape: (1716428, 17)
Installment shape: (1716428, 124)


In [4]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# bureau V1

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = trans_col = [
             'AMT_ANNUITY',
             'AMT_CREDIT_MAX_OVERDUE',
             'DAYS_ENDDATE_FACT',
             'AMT_CREDIT_SUM_LIMIT',
             'AMT_CREDIT_SUM_DEBT',
             'DAYS_CREDIT_ENDDATE',
             'AMT_CREDIT_SUM',
             'CREDIT_DAY_OVERDUE',
             'CNT_CREDIT_PROLONG',
             'AMT_CREDIT_SUM_OVERDUE',
             'DAYS_CREDIT_UPDATE']
for col in trans_col:
    print(col)
    X_bu[col] = qt.fit_transform(X_bu[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_CREDIT_MAX_OVERDUE
DAYS_ENDDATE_FACT
AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_SUM_DEBT
DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM
CREDIT_DAY_OVERDUE
CNT_CREDIT_PROLONG
AMT_CREDIT_SUM_OVERDUE
DAYS_CREDIT_UPDATE


In [6]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 1730000
#295
maxlen = 13

sorted_df = X_bu.sort_values(['SK_ID_CURR','DAYS_CREDIT'])
col = 'DAYS_CREDIT'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_BUREAU')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.bureau_rnn.value
report_file = ENV.bureau_report.value
pred_file = ENV.bureau_preds.value
pred_test_file = ENV.bureau_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 15
create document
get embedding Mat
...get ebd mat
num of words: 1716430
(1716430, 15)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/bureau/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5959567376944752
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5959567376944752
current roc is 0.6178756721625701
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.6471071848224688
current roc is 0.6464716181686189
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.6471071848224688
current roc is 0.6462973813636651
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.6471071848224688
current roc is 0.648987699496703
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 25 / 500
previous best roc is 0.648987699496703
current roc is 0.648

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 45 / 500
previous best roc is 0.6567109137824174
current roc is 0.6579516419561438
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.6579516419561438
current roc is 0.655801950204016
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.6579516419561438
current roc is 0.656337061910692
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.6579516419561438
current roc is 0.657

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.6591512064493174
current roc is 0.658979982539555
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.6591512064493174
current roc is 0.6582576767852121
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.6591512064493174
current roc is 0.659218752690085
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 73 / 500
previous best roc is 0.659218752690085
current roc is 0.6585

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 94 / 500
previous best roc is 0.6609344368898887
current roc is 0.6594941671783806
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.6609344368898887
current roc is 0.6586800344761292
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 96 / 500
previous best roc is 0.6609344368898887
current roc is 0.6584937514705798
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 97 / 500
previous best roc is 0.6609344368898887
current roc is 0.6592928699117939
Epoch 1/1
61503/61503 [==============================] - 0s 

...

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 196 / 500
previous best roc is 0.6658122861624935
current roc is 0.6647201839437856
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 197 / 500
previous best roc is 0.6658122861624935
current roc is 0.6641574513313074
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 198 / 500
previous best roc is 0.6658122861624935
current roc is 0.6645192449704964
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 199 / 500
previous best roc is 0.6658122861624935
current roc is 0.6648105401460686
Epoch 1/1
61502/61502 [==============================] -

saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 1 / 500
previous best roc is 0.6132176306988204
current roc is 0.6204509204306089
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 2 / 500
previous best roc is 0.6204509204306089
current roc is 0.6277977434963729
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 3 / 500
previous best roc is 0.6277977434963729
current roc is 0.629658728949149
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report

61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 45 / 500
previous best roc is 0.6543168512926364
current roc is 0.6545786041308668
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 46 / 500
previous best roc is 0.6545786041308668
current roc is 0.6550632395262196
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 47 / 500
previous best roc is 0.6550632395262196
current roc is 0.6549246847032004
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving r

61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 91 / 500
previous best roc is 0.6618133331678493
current roc is 0.6626886549393526
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 92 / 500
previous best roc is 0.6626886549393526
current roc is 0.662163860036003
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 93 / 500
previous best roc is 0.6626886549393526
current roc is 0.6613895138414726
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 94 / 500
previous best roc is 0.6626886549393526
current roc is 0.662202909326

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 116 / 500
previous best roc is 0.6642166744938831
current roc is 0.663129154655701
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 117 / 500
previous best roc is 0.6642166744938831
current roc is 0.6626569432125968
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 118 / 500
previous best roc is 0.6642166744938831
current roc is 0.6629270244926927
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 119 / 500
previous best roc is 0.6642166744938831
current roc is 0.6630518299731272
Epoch 1/1
61502/61502 [==============================] - 

In [7]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          4.908263
std           4.355940
min           1.000000
25%           1.000000
50%           4.000000
75%           7.000000
max         116.000000
Name: text, dtype: float64

In [8]:
aaaa

NameError: name 'aaaa' is not defined

# Bureau V2

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = [
             'AMT_ANNUITY',
             'AMT_CREDIT_MAX_OVERDUE',
             'DAYS_ENDDATE_FACT',
             'AMT_CREDIT_SUM_LIMIT',
             'AMT_CREDIT_SUM_DEBT',
             'DAYS_CREDIT_ENDDATE',
             'AMT_CREDIT_SUM',
             'CREDIT_DAY_OVERDUE',
             'CNT_CREDIT_PROLONG',
             'BUREAU_LENGTH',
             'AMT_CREDIT_SUM_OVERDUE',
             'DAYS_CREDIT_UPDATE',
    'AMT_ANNUITY_squre',
 'AMT_CREDIT_MAX_OVERDUE_squre',
 'DAYS_ENDDATE_FACT_squre',
 'AMT_CREDIT_SUM_LIMIT_squre',
 'AMT_CREDIT_SUM_DEBT_squre',
 'DAYS_CREDIT_ENDDATE_squre',
 'AMT_CREDIT_SUM_squre',
 'CREDIT_DAY_OVERDUE_squre',
 'CNT_CREDIT_PROLONG_squre',
 'BUREAU_LENGTH_squre',
 'AMT_CREDIT_SUM_OVERDUE_squre',
 'DAYS_CREDIT_UPDATE_squre',
    'AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE',
 'AMT_ANNUITY_DAYS_ENDDATE_FACT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_DEBT',
 'AMT_ANNUITY_DAYS_CREDIT_ENDDATE',
 'AMT_ANNUITY_AMT_CREDIT_SUM',
 'AMT_ANNUITY_CREDIT_DAY_OVERDUE',
 'AMT_ANNUITY_CNT_CREDIT_PROLONG',
 'AMT_ANNUITY_BUREAU_LENGTH',
 'AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE',
 'AMT_ANNUITY_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM',
 'AMT_CREDIT_MAX_OVERDUE_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_LENGTH',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_UPDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_LIMIT',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_DEBT',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM',
 'DAYS_ENDDATE_FACT_CREDIT_DAY_OVERDUE',
 'DAYS_ENDDATE_FACT_CNT_CREDIT_PROLONG',
 'DAYS_ENDDATE_FACT_BUREAU_LENGTH',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_LIMIT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_LIMIT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_DEBT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM',
 'DAYS_CREDIT_ENDDATE_CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE_CNT_CREDIT_PROLONG',
 'DAYS_CREDIT_ENDDATE_BUREAU_LENGTH',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_ENDDATE_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DAYS_CREDIT_UPDATE',
 'CREDIT_DAY_OVERDUE_CNT_CREDIT_PROLONG',
 'CREDIT_DAY_OVERDUE_BUREAU_LENGTH',
 'CREDIT_DAY_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_DAY_OVERDUE_DAYS_CREDIT_UPDATE',
 'CNT_CREDIT_PROLONG_BUREAU_LENGTH',
 'CNT_CREDIT_PROLONG_AMT_CREDIT_SUM_OVERDUE',
 'CNT_CREDIT_PROLONG_DAYS_CREDIT_UPDATE',
 'BUREAU_LENGTH_AMT_CREDIT_SUM_OVERDUE',
 'BUREAU_LENGTH_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_OVERDUE_DAYS_CREDIT_UPDATE']
for col in trans_col:
    print(col)
    X_bu_fe[col] = qt.fit_transform(X_bu_fe[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_CREDIT_MAX_OVERDUE
DAYS_ENDDATE_FACT
AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_SUM_DEBT
DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM
CREDIT_DAY_OVERDUE
CNT_CREDIT_PROLONG
BUREAU_LENGTH
AMT_CREDIT_SUM_OVERDUE
DAYS_CREDIT_UPDATE
AMT_ANNUITY_squre
AMT_CREDIT_MAX_OVERDUE_squre
DAYS_ENDDATE_FACT_squre
AMT_CREDIT_SUM_LIMIT_squre
AMT_CREDIT_SUM_DEBT_squre
DAYS_CREDIT_ENDDATE_squre
AMT_CREDIT_SUM_squre
CREDIT_DAY_OVERDUE_squre
CNT_CREDIT_PROLONG_squre
BUREAU_LENGTH_squre
AMT_CREDIT_SUM_OVERDUE_squre
DAYS_CREDIT_UPDATE_squre
AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE
AMT_ANNUITY_DAYS_ENDDATE_FACT
AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT
AMT_ANNUITY_AMT_CREDIT_SUM_DEBT
AMT_ANNUITY_DAYS_CREDIT_ENDDATE
AMT_ANNUITY_AMT_CREDIT_SUM
AMT_ANNUITY_CREDIT_DAY_OVERDUE
AMT_ANNUITY_CNT_CREDIT_PROLONG
AMT_ANNUITY_BUREAU_LENGTH
AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE
AMT_ANNUITY_DAYS_CREDIT_UPDATE
AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT
AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT
AMT_CREDIT_

In [7]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 1730000
#295
maxlen = 13

sorted_df = X_bu_fe.sort_values(['SK_ID_CURR','DAYS_CREDIT'])
col = 'DAYS_CREDIT'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_BUREAU')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.bureau_rnn.value
report_file = ENV.bureau_report.value
pred_file = ENV.bureau_preds.value
pred_test_file = ENV.bureau_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 122
create document
get embedding Mat
...get ebd mat
num of words: 1716430
(1716430, 122)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/bureau/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.6101660302523371
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.6101660302523371
current roc is 0.6262922971030296
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.p

61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 45 / 500
previous best roc is 0.6627320825992391
current roc is 0.6611197962995908
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.6627320825992391
current roc is 0.6621830272853527
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.6627320825992391
current roc is 0.6621426598699577
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.6627320825992391
current roc is 0.66276256126

61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 69 / 500
previous best roc is 0.6635246569603617
current roc is 0.6613608189479545
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.6635246569603617
current roc is 0.6608979467836626
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.6635246569603617
current roc is 0.6598394090995173
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.6635246569603617
current roc is 0.6614960064791594
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
s

61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_1.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 1 / 500
previous best roc is 0.6057125892129902
current roc is 0.6160188301032818
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_1.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 2 / 500
previous best roc is 0.6160188301032818
current roc is 0.6267563485205845
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_1.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 3 / 500
previous best roc is 0.6267563485205845
current roc is 0.6360500802217345
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving mode

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 22 / 500
previous best roc is 0.6580788345977949
current roc is 0.6573352617848998
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_1.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 23 / 500
previous best roc is 0.6580788345977949
current roc is 0.6580929809842465
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 24 / 500
previous best roc is 0.6580929809842465
current roc is 0.6561761168778525
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_1.pkl
======= current 25 / 500
previous best roc is 0.6580929809842465
current roc is 0.65744677628

...

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 57 / 500
previous best roc is 0.6708454700355031
current roc is 0.6691961228942467
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 58 / 500
previous best roc is 0.6708454700355031
current roc is 0.6700571172733303
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 59 / 500
previous best roc is 0.6708454700355031
current roc is 0.6693852593057033
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 60 / 500
previous best roc is 0.6708454700355031
current roc is 0.6694615641570665
Epoch 1/1
61502/61502 [==============================] - 0s 

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 82 / 500
previous best roc is 0.6714571585715542
current roc is 0.6703266514755235
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 83 / 500
previous best roc is 0.6714571585715542
current roc is 0.6703093182532688
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 84 / 500
previous best roc is 0.6714571585715542
current roc is 0.6679554125998901
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 85 / 500
previous best roc is 0.6714571585715542
current roc is 0.6699002463301365
Epoch 1/1
61502/61502 [==============================] - 0s 

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 107 / 500
previous best roc is 0.6716918311715865
current roc is 0.6701410995192837
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 108 / 500
previous best roc is 0.6716918311715865
current roc is 0.6700848421929267
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 109 / 500
previous best roc is 0.6716918311715865
current roc is 0.6706735558391966
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_3.pkl
======= current 110 / 500
previous best roc is 0.6716918311715865
current roc is 0.6703113235429006
Epoch 1/1
61502/61502 [==============================] -

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 15 / 500
previous best roc is 0.6594872243075994
current roc is 0.6594611305723497
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 16 / 500
previous best roc is 0.6594872243075994
current roc is 0.6603650503978139
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 17 / 500
previous best roc is 0.6603650503978139
current roc is 0.6625743061196451
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 1

61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 62 / 500
previous best roc is 0.6687650958885372
current roc is 0.6693724139699802
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 63 / 500
previous best roc is 0.6693724139699802
current roc is 0.6696555394054422
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/bureau/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/bureau/report_fold_4.pkl
======= current 64 / 500
previous best roc is 0.6696555394054422
current roc is 0.6697528914050163
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving m

In [10]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          4.908263
std           4.355940
min           1.000000
25%           1.000000
50%           4.000000
75%           7.000000
max         116.000000
Name: text, dtype: float64

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()